In [1]:
import geopandas as gpd
import pygeos, os, glob
from sqlalchemy import create_engine

In [2]:
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

In [3]:
sql = '''
SELECT 
OBJECTID AS id,
name,
type,
fed_class,
Shape.STAsBinary() AS geometry
FROM dbo.Road;
'''

In [4]:
roads = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geometry' )

In [5]:
roads.crs = "EPSG:2914"

In [6]:
roads.head()

,id,name,type,fed_class,geometry
0,1,FRANKLIN SCHOOL,RD,Local,"LINESTRING (4183480.518 925529.728, 4183475.26..."
1,2,ALCORN,ST,Local,"LINESTRING (4301176.027 922365.878, 4301184.02..."
2,3,WALLING,ST,Local,"LINESTRING (4303049.526 922151.132, 4303037.02..."
3,4,RAILROAD,LN,Local,"LINESTRING (4300168.023 921087.826, 4300329.02..."
4,5,PETE NEWELL,DR,Local,"LINESTRING (4467432.695 918795.907, 4467636.19..."


In [7]:
path = r"T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles"

In [8]:
shpfiles = glob.glob(os.path.join(path, "2020_BaseNetwork", "*.shp"))

In [9]:
shpfiles.extend(glob.glob(os.path.join(path, "2045_RTP_Network_Shapefiles", "*.shp")))

In [10]:
shpfiles

['T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2020_BaseNetwork\\2020_emme_links.shp',
 'T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2020_BaseNetwork\\2020_emme_nodes.shp',
 'T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2020_BaseNetwork\\2020_emme_tlines.shp',
 'T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2045_RTP_Network_Shapefiles\\2045_emme_links.shp',
 'T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2045_RTP_Network_Shapefiles\\2045_emme_nodes.shp',
 'T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2045_RTP_Network_Shapefiles\\emme_tlines_OffPeak.shp',
 'T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2045_RTP_Network_Shapefiles\\emme_tlines_Peak.shp',
 'T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2045_RTP_Network_Shapefiles\\emme_tsegs_OffPeak.shp',
 'T:\\DCProjects\\Modeling\\KateModel\\requests\\2022021

In [11]:
file = shpfiles[0]

In [12]:
file

'T:\\DCProjects\\Modeling\\KateModel\\requests\\20220210 lane miles\\2020_BaseNetwork\\2020_emme_links.shp'

In [13]:
gdf = gpd.read_file(file)

In [14]:
roads = roads.to_crs(gdf.crs)

In [15]:
gdf.crs

<Projected CRS: PROJCS["NAD27 / Oregon South",GEOGCS["NAD27",DATUM ...>
Name: NAD27 / Oregon South
Axis Info [cartesian]:
- [east]: Easting (US survey mile)
- [north]: Northing (US survey mile)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1927
- Ellipsoid: Clarke 1866
- Prime Meridian: Greenwich

In [16]:
print(gdf.crs)

PROJCS["NAD27 / Oregon South",GEOGCS["NAD27",DATUM["North_American_Datum_1927",SPHEROID["Clarke 1866",6378206.4,294.9786982,AUTHORITY["EPSG","7008"]],AUTHORITY["EPSG","6267"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AUTHORITY["EPSG","4267"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",41.6666666666667],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",42.3333333333333],PARAMETER["standard_parallel_2",44],PARAMETER["false_easting",378.8439095681],PARAMETER["false_northing",0.006818182],UNIT["US survey mile",1609.34721869444,AUTHORITY["EPSG","9035"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [17]:
def addFunctionalClass(file, export=True):
    gdf = gpd.read_file(file)
    sjoin = gpd.sjoin_nearest(gdf, roads, distance_col="distances")
    if file.find('nodes')==-1:
        sjoin['miles'] = sjoin.geometry.length
    if export:
        sjoin.drop(columns=['index_right', 'id', 'name', 'type'], inplace=True)
        sjoin.to_file(os.path.join(path, 'output', file.split('\\')[-1]))
         
    return sjoin

In [18]:
sjoin = addFunctionalClass(file, export=False)

In [19]:
sjoin.head()

,ID,INODE,JNODE,LENGTH,TYPE,MODES,LANES,VDF,DATA1,DATA2,...,@wlen,@xland,geometry,index_right,id,name,type,fed_class,distances,miles
0,1-2519,1,2519,0.4,9,awp,1.0,20,189.173030,115.521740,...,0.4,0.0,"LINESTRING (249.737 173.779, 249.720 173.375)",14548,14578,GREEN ISLAND,RD,Local,0.000000,0.404100
5286,2518-2519,2518,2519,0.5,7,abwxp,1.0,5,93.754204,50.903637,...,0.5,0.0,"LINESTRING (249.700 172.877, 249.720 173.375)",14548,14578,GREEN ISLAND,RD,Local,0.007781,0.498375
5288,2519-1,2519,1,0.4,9,awp,1.0,20,121.205310,75.419678,...,0.4,0.0,"LINESTRING (249.720 173.375, 249.737 173.779)",14548,14578,GREEN ISLAND,RD,Local,0.000000,0.404100
5289,2519-2518,2519,2518,0.5,7,abwxp,1.0,5,75.745468,43.332863,...,0.5,0.0,"LINESTRING (249.720 173.375, 249.700 172.877)",14548,14578,GREEN ISLAND,RD,Local,0.007781,0.498375
1,2-1822,2,1822,0.4,9,awp,2.0,20,2685.900100,1562.664300,...,0.4,0.0,"LINESTRING (251.613 174.981, 251.631 174.584)",20858,20916,WILKINS,RD,Local,0.000000,0.396908


In [20]:
for file in shpfiles:
    sjoin = addFunctionalClass(file)
    print(file)

T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2020_BaseNetwork\2020_emme_links.shp
T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2020_BaseNetwork\2020_emme_nodes.shp
T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2020_BaseNetwork\2020_emme_tlines.shp
T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2045_RTP_Network_Shapefiles\2045_emme_links.shp
T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2045_RTP_Network_Shapefiles\2045_emme_nodes.shp
T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2045_RTP_Network_Shapefiles\emme_tlines_OffPeak.shp
T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2045_RTP_Network_Shapefiles\emme_tlines_Peak.shp
T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2045_RTP_Network_Shapefiles\emme_tsegs_OffPeak.shp
T:\DCProjects\Modeling\KateModel\requests\20220210 lane miles\2045_RTP_Network_Shapefiles\emme_tsegs_Peak.shp
